In [ ]:
!pip install kaggle


In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json file


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kavyaprabahar","key":"2cd0226074ba1144bb4c8210ca11acfe"}'}

In [ ]:
!kaggle datasets download -d umeradnaan/x-ray-dection

Dataset URL: https://www.kaggle.com/datasets/umeradnaan/x-ray-dection
License(s): CC0-1.0
 96% 166M/173M [00:02<00:00, 83.7MB/s]
100% 173M/173M [00:02<00:00, 89.0MB/s]


In [ ]:
!unzip x-ray-dection.zip

Streaming output truncated to the last 5000 lines.
  inflating: Bone Fracture Dataset/training/fractured/84-rotated2-rotated3-rotated3.jpg  
  inflating: Bone Fracture Dataset/training/fractured/84-rotated2-rotated3.jpg  
  inflating: Bone Fracture Dataset/training/fractured/84-rotated2.jpg  
  inflating: Bone Fracture Dataset/training/fractured/84-rotated3-rotated1-rotated1.jpg  
  inflating: Bone Fracture Dataset/training/fractured/84-rotated3-rotated1-rotated2.jpg  
  inflating: Bone Fracture Dataset/training/fractured/84-rotated3-rotated1-rotated3.jpg  
  inflating: Bone Fracture Dataset/training/fractured/84-rotated3-rotated1.jpg  
  inflating: Bone Fracture Dataset/training/fractured/84-rotated3-rotated2-rotated1.jpg  
  inflating: Bone Fracture Dataset/training/fractured/84-rotated3-rotated2-rotated2.jpg  
  inflating: Bone Fracture Dataset/training/fractured/84-rotated3-rotated2-rotated3.jpg  
  inflating: Bone Fracture Dataset/training/fractured/84-rotated3-rotated2.jpg  
  in

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

base_dir = '/content/Bone Fracture Dataset'  # Replace with the path to your dataset
train_dir = os.path.join(base_dir, 'training')
val_dir = os.path.join(base_dir, 'testing')


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [ ]:
img_height, img_width = 224, 224  # Image size for EfficientNetB0

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='binary'
)


Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Adjust epochs as needed
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


276/276 ━━━━━━━━━━━━━━━━━━━━ 875s 3s/step - accuracy: 0.5031 - loss: 0.6983 - val_accuracy: 0.3976 - val_loss: 0.6935
Epoch 2/10
  1/276 ━━━━━━━━━━━━━━━━━━━━ 9:35 2s/step - accuracy: 0.4062 - loss: 0.6935

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


276/276 ━━━━━━━━━━━━━━━━━━━━ 36s 123ms/step - accuracy: 0.4062 - loss: 0.6935 - val_accuracy: 0.4583 - val_loss: 0.6933
Epoch 3/10
276/276 ━━━━━━━━━━━━━━━━━━━━ 849s 3s/step - accuracy: 0.5056 - loss: 0.6932 - val_accuracy: 0.6024 - val_loss: 0.6917
Epoch 4/10
276/276 ━━━━━━━━━━━━━━━━━━━━ 39s 129ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5417 - val_loss: 0.6926
Epoch 5/10
276/276 ━━━━━━━━━━━━━━━━━━━━ 817s 3s/step - accuracy: 0.5013 - loss: 0.6933 - val_accuracy: 0.6076 - val_loss: 0.6920
Epoch 6/10
276/276 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5312 - loss: 0.6928 - val_accuracy: 0.4167 - val_loss: 0.6941
Epoch 7/10
276/276 ━━━━━━━━━━━━━━━━━━━━ 831s 3s/step - accuracy: 0.5131 - loss: 0.6930 - val_accuracy: 0.5972 - val_loss: 0.6916
Epoch 8/10
276/276 ━━━━━━━━━━━━━━━━━━━━ 36s 121ms/step - accuracy: 0.5312 - loss: 0.6927 - val_accuracy: 0.6667 - val_loss: 0.6905
Epoch 9/10
276/276 ━━━━━━━━━━━━━━━━━━━━ 803s 3s/step - accuracy: 0.5121 - loss: 0.6930 - val_accuracy: 

In [ ]:
model.save('/content/drive/MyDrive/spinexray.h5')

In [ ]:
model.evaluate(val_generator)

19/19 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.6145 - loss: 0.6912


[0.6914312839508057, 0.6000000238418579]

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the model
model = tf.keras.models.load_model('/content/drive/MyDrive/spinexray.h5')

# Load and preprocess the image
img_path = '/content/drive/MyDrive/images.jpeg'  # Replace with the path to your image
img = image.load_img(img_path, target_size=(224, 224))  # Match target size used during training
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Match preprocessing used during training

# Make prediction
prediction = model.predict(img_array)

# Interpret the prediction
if prediction[0] > 0.5:
    print("The image is predicted to show a fracture.")
else:
    print("The image is predicted to show no fracture.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
The image is predicted to show no fracture.
